In [2]:
# Importando os bibliotecas/pacotes
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## Web Scraping dos Dados Steamdb

In [4]:
# Configurar o serviço do ChromeDriver
servico = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=servico)

# Navegar até a página
driver.get('https://steamdb.info/sales/')

# Esperar até que a tabela de vendas seja carregada
wait = WebDriverWait(driver, 10)
table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.table-sales')))

# Extrair os cabeçalhos da tabela
headers = [header.text for header in table.find_elements(By.CSS_SELECTOR, 'thead th')]

# Função para extrair dados de uma página
def extract_data():
    rows = table.find_elements(By.CSS_SELECTOR, 'tbody tr')
    page_data = []
    for row in rows:
        name = row.find_element(By.CSS_SELECTOR, '.b').text  # Pega o primeiro nome na coluna Name
        cells = [cell.text for cell in row.find_elements(By.CSS_SELECTOR, 'td')]
        page_data.append([name] + cells[1:])  # Adiciona o nome uma vez e as demais colunas
    return page_data

# Inicializar lista de dados
data = []

# Extrair dados de todas as páginas
for page_num in range(1, 10):  # Supondo que existam 9 páginas
    print(f"Extraindo dados da página {page_num}")
    
    # Esperar até que a tabela de vendas seja carregada
    table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.table-sales')))
    data.extend(extract_data())
    
    # Pausar para evitar sobrecarga do servidor
    time.sleep(2)
    
    # Tentar ir para a próxima página se não estivermos na última página
    if page_num < 9:
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.dt-paging-button')))
            if next_button.is_displayed() and next_button.is_enabled():
                next_button.click()
                # Esperar um curto período para garantir que a página carregue
                time.sleep(2)
            else:
                print("Botão 'Next' não está visível ou habilitado.")
                break
        except Exception as e:
            print("Erro ao tentar ir para a próxima página:", e)
            break

# Fechar o navegador
driver.quit()

# Ajustar os cabeçalhos da tabela para refletir as colunas extraídas
df = pd.DataFrame(data, columns=['Name'] + headers[1:]) 

# Garantir que os nomes das colunas sejam únicos
df.columns = pd.Index([f'{col}_{i}' if list(df.columns).count(col) > 1 else col for i, col in enumerate(df.columns)])

# Salvar o DataFrame como JSON
df.to_json('steamdb_sales.json', orient='records', lines=True)

# Exibir o DataFrame para verificar os dados
print(df.head())

Extraindo dados da página 1
Extraindo dados da página 2
Extraindo dados da página 3
Extraindo dados da página 4
Extraindo dados da página 5
Extraindo dados da página 6
Extraindo dados da página 7
Extraindo dados da página 8
Extraindo dados da página 9
              Name_0                                               Name_2  \
0  Machinika: Museum    Machinika: Museum\nFree To Keep\nnew historica...   
1    ENDLESS™ Legend    ENDLESS™ Legend\nFree To Keep\nnew historical low   
2  100% Orange Juice                      100% Orange Juice\nFree To Keep   
3    Killing Floor 2    Killing Floor 2\nPlay For Free\nnew historical...   
4  Wizard with a Gun    Wizard with a Gun\nPlay For Free\nnew historic...   

       %     Price  Rating   Release       Ends     Started  
0  -100%   R$ 0,00  87.44%  Mar 2021  in 7 days  7 days ago  
1  -100%   R$ 0,00  82.27%  Sep 2014  in 3 days  4 days ago  
2  -100%   R$ 0,00  90.40%  Sep 2013  in 2 days  4 days ago  
3   -95%   R$ 2,79  86.60%  Nov 2016 

In [5]:
# Carregar o arquivo JSON para um DataFrame
df = pd.read_json('steamdb_sales.json', orient='records', lines=True)
df

,Name_0,,Name_2,%,Price,Rating,Release,Ends,Started
0,Machinika: Museum,,Machinika: Museum\nFree To Keep\nnew historica...,-100%,"R$ 0,00",87.44%,Mar 2021,in 7 days,7 days ago
1,ENDLESS™ Legend,,ENDLESS™ Legend\nFree To Keep\nnew historical low,-100%,"R$ 0,00",82.27%,Sep 2014,in 3 days,4 days ago
2,100% Orange Juice,,100% Orange Juice\nFree To Keep,-100%,"R$ 0,00",90.40%,Sep 2013,in 2 days,4 days ago
3,Killing Floor 2,,Killing Floor 2\nPlay For Free\nnew historical...,-95%,"R$ 2,79",86.60%,Nov 2016,in 3 days,4 days ago
4,Wizard with a Gun,,Wizard with a Gun\nPlay For Free\nnew historic...,-50%,"R$ 29,99",78.06%,Oct 2023,in 3 days,7 days ago
...,...,...,...,...,...,...,...,...,...
895,Being a DIK - Season 1,,Being a DIK - Season 1,-25%,"R$ 20,91",93.84%,Feb 2020,in 9 days,5 days ago
896,Valheim,,Valheim,-50%,"R$ 18,99",93.82%,Feb 2021,in 8 days,6 days ago
897,My Friend Pedro,,My Friend Pedro,-75%,"R$ 14,99",93.64%,Jun 2019,in 3 days,4 days ago
898,月影魅像-解放之羽-,,月影魅像-解放之羽-,-65%,"R$ 23,09",93.52%,Jul 2019,in 6 days,2 days ago


## Transformando os dados

In [7]:
# Excluir a coluna Name_2 e coluna vazia ""
df.drop('Name_2', axis=1, inplace=True)
df.drop('', axis=1, inplace=True)

In [8]:
# Renomear a coluna Name_0 para Name e % por Percentage
df.rename(columns={'Name_0': 'Name'}, inplace=True)
df.rename(columns={'%': 'Percentage'}, inplace=True)

In [9]:
# Remover caracteres indesejados e converter para o formato correto
df['Price'] = df['Price'].str.replace('R$', '').str.replace(',', '.').astype(float)
df['Release'] = pd.to_datetime(df['Release'], format='%b %Y', errors='coerce').dt.strftime('%Y-%m')
df['Rating'] = df['Rating'].str.replace('%', '').astype(float)
df['Percentage'] = df['Percentage'].str.replace('%', '').str.replace('-', '').astype(float)

In [10]:
# verificar valores nulos
print(df.isnull().sum())

Name          0
Percentage    0
Price         0
Rating        0
Release       0
Ends          0
Started       0
dtype: int64


In [11]:
df

,Name,Percentage,Price,Rating,Release,Ends,Started
0,Machinika: Museum,100.0,0.00,87.44,2021-03,in 7 days,7 days ago
1,ENDLESS™ Legend,100.0,0.00,82.27,2014-09,in 3 days,4 days ago
2,100% Orange Juice,100.0,0.00,90.40,2013-09,in 2 days,4 days ago
3,Killing Floor 2,95.0,2.79,86.60,2016-11,in 3 days,4 days ago
4,Wizard with a Gun,50.0,29.99,78.06,2023-10,in 3 days,7 days ago
...,...,...,...,...,...,...,...
895,Being a DIK - Season 1,25.0,20.91,93.84,2020-02,in 9 days,5 days ago
896,Valheim,50.0,18.99,93.82,2021-02,in 8 days,6 days ago
897,My Friend Pedro,75.0,14.99,93.64,2019-06,in 3 days,4 days ago
898,月影魅像-解放之羽-,65.0,23.09,93.52,2019-07,in 6 days,2 days ago


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 900 entries, 0 to 899
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Name        900 non-null    object 
 1   Percentage  900 non-null    float64
 2   Price       900 non-null    float64
 3   Rating      900 non-null    float64
 4   Release     900 non-null    object 
 5   Ends        900 non-null    object 
 6   Started     900 non-null    object 
dtypes: float64(3), object(4)
memory usage: 56.2+ KB


In [13]:
# Salvar o DataFrame como JSON
df.to_json('steamdb_sales_modificado.json', orient='records', lines=True)

In [14]:
df = pd.read_json('steamdb_sales_modificado.json', orient='records', lines=True)
df

,Name,Percentage,Price,Rating,Release,Ends,Started
0,Machinika: Museum,100,0.00,87.44,2021-03,in 7 days,7 days ago
1,ENDLESS™ Legend,100,0.00,82.27,2014-09,in 3 days,4 days ago
2,100% Orange Juice,100,0.00,90.40,2013-09,in 2 days,4 days ago
3,Killing Floor 2,95,2.79,86.60,2016-11,in 3 days,4 days ago
4,Wizard with a Gun,50,29.99,78.06,2023-10,in 3 days,7 days ago
...,...,...,...,...,...,...,...
895,Being a DIK - Season 1,25,20.91,93.84,2020-02,in 9 days,5 days ago
896,Valheim,50,18.99,93.82,2021-02,in 8 days,6 days ago
897,My Friend Pedro,75,14.99,93.64,2019-06,in 3 days,4 days ago
898,月影魅像-解放之羽-,65,23.09,93.52,2019-07,in 6 days,2 days ago
